In [ ]:
pip install -q -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.7/217.7 kB 8.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
import json, os
import google.generativeai as genai

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. read from secret file
with open('/content/drive/MyDrive/secrets/api_keys.json', encoding='utf-8-sig') as f:
    secrets = json.load(f)

# 3. get the keys
SERP_KEY   = secrets["SERPAPI_API_KEY"]
GEMINI_KEY = secrets["GEMINI_API_KEY"]

# 4. Configure the Gemini API
genai.configure(api_key=GEMINI_KEY)

# 5. Define the model
model = genai.GenerativeModel(model_name="gemini-2.5-flash")

# 6. example
response = model.generate_content("Explain how AI works in a few words")
print(response.text)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
AI learns from data to recognize patterns and make predictions or decisions.


In [ ]:

import os, requests, bs4

def fetch_page_text(url):
    try:
        res = requests.get(url, timeout=10)
        soup = bs4.BeautifulSoup(res.text, "html.parser")
        return soup.get_text(separator=" ", strip=True)
    except Exception as e:
        print("Fetch error:", e)
        return ""


In [ ]:
import os
import requests

def get_google_links_direct(query, num_results=3):
    api_key = os.getenv("SERPAPI_API_KEY")
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key,
    }
    response = requests.get("https://serpapi.com/search", params=params)
    data = response.json()
    results = data.get("organic_results", [])
    links = [r['link'] for r in results[:num_results]]
    return links

# 🔍 1. Tender search
tender_query = "security camera procurement tender site:europa.eu"
tender_links = get_google_links_direct(tender_query)
print("🔗 Tender links:", tender_links)

# 🛒 2. Supplier search
supplier_query = "security camera supplier Europe price range site:.com OR site:.co.uk"
supplier_links = get_google_links_direct(supplier_query)
print("🛒 Supplier Links:", supplier_links)

# 🔄 3. Merge All
all_links = tender_links + supplier_links
print("🔗 All links:", all_links)


🔗 Tender links: ['https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/dba52083-8af6-4ab4-82a2-d7192aa0a0f7', 'https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/37a10779-0375-4408-9236-afe3bcd36f6a', 'https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/0d540a61-b646-4cef-97c1-e6a97ad74721']
🛒 Supplier Links: ['https://uk.swann.com/products/?srsltid=AfmBOopChlfgI8r8CFY2R_v9a9QlRGxWW9tGsyy0EFSXs3VNPdgdpclA', 'https://www.eufy.com/collections/security-camera', 'https://www.kentfaith.com/KF50.0006AEU_outdoor-security-camera-2k-wireless-camera-for-home-security-2-4ghz-wifi-security-camera-with-auto-tracking-motion-detection-with-spotlight-color-night-vision-camera-2-way-audio-euro-gauge?srsltid=AfmBOoqgLrty_CZWoTQSocAhtoRvdl0RRuVRkMHiZHZkmT5NLoj_cEmQ']
🔗 All links: ['https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/dba52083-8af6-4ab4-82a2-d7192aa0a0f7', 'https://op.europa.

In [ ]:
import os, requests, bs4
def fetch_page_text(url):
    try:
        res = requests.get(url, timeout=10)
        soup = bs4.BeautifulSoup(res.text, "html.parser")
        return soup.get_text(separator=" ", strip=True)
    except Exception as e:
        print("Fetch error:", e)
        return ""

tender_texts   = [fetch_page_text(u) for u in tender_links]
supplier_texts = [fetch_page_text(u) for u in supplier_links]


In [ ]:
from google import genai

# Initialize Gemini Client

# Initialize Gemini Client
client = genai.Client(api_key=GEMINI_KEY)

# Construct prompts from the tender and supplier data
tender_prompt = f"""
You are a public tender expert for Dublin City Council.

Below are documents from security camera public tenders:
{tender_texts[0][:1500]}
{tender_texts[1][:1500] if len(tender_texts) > 1 else ''}

1. Summarize the most common **technical and legal requirements**.
2. Generate 5 clarification questions that a city official should answer before drafting a new tender.
"""

supplier_prompt = f"""
You are a market analyst advising a municipality about camera system purchases.

Below are supplier websites or documents:
{supplier_texts[0][:1500]}
{supplier_texts[1][:1500] if len(supplier_texts) > 1 else ''}

1. Summarize the most common **features, pricing, and delivery models** offered in the market.
2. Generate 5 clarifying questions to help compare or choose between supplier options.
"""

# Generate content using Gemini 2 Flash
tender_resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=tender_prompt
)

supplier_resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=supplier_prompt
)

# Output the results
print("📑 Tender Summary + Questions:\n")
print(tender_resp.text)

print("\n🛒 Supplier Summary + Questions:\n")
print(supplier_resp.text)


# Construct prompts from the tender and supplier data
tender_prompt = f"""
You are a public tender expert for Dublin City Council.

Below are documents from security camera public tenders:
{tender_texts[0][:1500]}
{tender_texts[1][:1500] if len(tender_texts) > 1 else ''}

1. Summarize the most common **technical and legal requirements**.
2. Generate 5 clarification questions that a city official should answer before drafting a new tender.
"""

supplier_prompt = f"""
You are a market analyst advising a municipality about camera system purchases.

Below are supplier websites or documents:
{supplier_texts[0][:1500]}
{supplier_texts[1][:1500] if len(supplier_texts) > 1 else ''}

1. Summarize the most common **features, pricing, and delivery models** offered in the market.
2. Generate 5 clarifying questions to help compare or choose between supplier options.
"""

# Generate content using Gemini 2 Flash
tender_resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=tender_prompt
)

supplier_resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=supplier_prompt
)

# Output the results
print("📑 Tender Summary + Questions:\n")
print(tender_resp.text)

print("\n🛒 Supplier Summary + Questions:\n")
print(supplier_resp.text)


📑 Tender Summary + Questions:

As a public tender expert for Dublin City Council, based on the provided partial documents from the Publications Office of the EU, here's a summary of inferred requirements and clarification questions for a new tender:

## Summary of Most Common Technical and Legal Requirements (Inferred)

The provided snippets are primarily navigation and header information from the EU's official procurement portal. While they don't list specific clauses, they strongly imply the following:

### 1. Technical Requirements:
*   **Focus on Video Surveillance:** The tenders explicitly relate to "Camera surveillance," "video surveillance," and "video protection cameras."
*   **Digital IP Technology:** A specific emphasis on "Digital cameras (IP)" indicates that the tender will likely require modern, network-based camera systems, as opposed to older analog systems. This implies requirements for network connectivity, potentially Power over Ethernet (PoE), and integration with ne

In [ ]:
# -----------------------------
#  EXAMPLE ANSWERS
# -----------------------------
user_answers = """
TENDER-SIDE CLARIFICATION ANSWERS
1. Purpose & Scope
- Main goals: crime prevention around parks and municipal buildings, plus traffic-flow monitoring at busy intersections.
- Coverage: 10 public parks, 5 key intersections, 3 municipal buildings (≈60 cameras total).

2. Technical requirements and integration
- Specs: 4K, PTZ, low-light (≤0.01 lux), IP66.
- Required features: motion detection, ANPR. Optional: facial recognition.
- Must integrate with Milestone VMS and existing city network.

3. Data & Privacy
- Footage kept 30 days, encrypted at rest and in transit.
- GDPR Article 35 DPIA will be completed; local signage required under DCC by-laws.

4. Budget and Contract Structure
- Max budget: €120,000 (excl. VAT).
- 3-year fixed-price contract, with 2-year extension option.
- Funded via Urban Safety & EU grants.

5. Evaluation Criteria and Weighting
- Technical (40%), price (25%), SLA/support (15%), experience (10%), sustainability (10%).
- Bonus: local suppliers, ISO-certified, low-energy hardware.

SUPPLIER-SIDE CLARIFICATION ANSWERS
1. Total Cost of Ownership
Q: What’s the 5-year total cost (hardware, install, support, storage, replacements)?
A: ~€95,000 over 5 years for 60 units. Includes PoE install, 24/7 support, 14-day cloud retention.

2. Warranty & Support
Q: What’s your warranty and repair/support response time?
A: 3-year warranty. Remote diagnostics within 2h; on-site repair in 24–48h, incl. rural areas.

3. Cybersecurity
Q: How do you address security (encryption, patching, access control)?
A: AES-256 encryption, auto firmware updates, 2FA for admin access. ISO 27001 certified.

4. System Integration
Q: Can your system connect with our VMS and emergency comms?
A: Compatible with ONVIF and SIP systems. Offers REST APIs and modular UI extensions.

5. Power Reliability
Q: What’s your battery/solar performance in Irish weather?
A: Solar cameras rated for 4.5h/day sunlight. Battery backup lasts 72h; AC fallback optional.
"""

#Gemini Prompt
draft_prompt = f"""
You are a public procurement expert working at Dublin City Council.
You have conducted a market analysis on municipal surveillance camera systems and identified both common supplier offerings and typical tender requirements.

Now write a **complete and professional 'Requirements and Specifications' section** for a new security camera tender.
Structure it just like the real Irish tender documents provided before. Include the following:

- A short introduction / background
- Detailed technical requirements in bullet points or tables (e.g., resolution, night vision, weatherproofing, storage, analytics)
- Legal and compliance requirements (especially GDPR and EU procurement directives)
- Installation and maintenance responsibilities
- Service levels (SLAs), warranties, and support
- Any environmental or sustainability standards
- Pricing and contractual expectations

Make sure the structure and language matches what is commonly used in Irish public sector tenders.
Do not generate questions. Generate a final tender section text.
Length: at least 700–1000 words.


--- USER NOTES ---
{user_answers}
"""
# Generate content (⚠️ only `model` and `contents` supported)
draft_resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=draft_prompt
)

# Safely extract and print response
try:
    print("📄 FINAL REQUIREMENTS & SPECIFICATIONS\n")
    print(draft_resp.text)
except AttributeError:
    print(draft_resp.candidates[0].content.parts[0].text)

📄 FINAL REQUIREMENTS & SPECIFICATIONS

**Section 3: Requirements and Specifications**

**3.1 Introduction and Background**
Dublin City Council (DCC) seeks to procure, install, and maintain a state-of-the-art municipal surveillance camera system to enhance public safety and operational efficiency across designated areas within the Dublin administrative region. The primary objectives of this procurement are:
*   **Crime Prevention and Public Safety:** To deter illicit activities, improve incident response, and enhance the safety of citizens within public spaces, particularly around municipal parks and buildings.
*   **Traffic Management and Flow Optimisation:** To monitor traffic patterns, manage congestion, and support urban planning initiatives at key arterial intersections.

The scope of this tender encompasses the supply, installation, configuration, and ongoing maintenance of approximately sixty (60) high-definition surveillance camera units. These units shall be strategically deplo

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Car Park Management Solution ITT_Final.pdf to Car Park Management Solution ITT_Final.pdf
Saving Final Gully Monitoring CFT.pdf to Final Gully Monitoring CFT.pdf
Saving RFT for Service Design.pdf to RFT for Service Design.pdf


In [ ]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

tender_texts = []
for filename in uploaded.keys():
    reader = PdfReader(filename)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text
    tender_texts.append(text)


In [ ]:
draft_prompt = f"""
You are a senior tender documentation officer working at Dublin City Council.
You are responsible for writing detailed and high-standard tender documents in accordance with Irish and EU procurement norms.

Below are authentic excerpts from previously published public tenders related to security technologies:
{tender_texts[0][:1500]}
{tender_texts[1][:1500] if len(tender_texts) > 1 else ''}

You have also received internal notes clarifying specific needs and expectations for this procurement:
--- USER CLARIFICATIONS ---
{user_answers}

Your task is to write an extremely comprehensive **"Requirements and Specifications"** section for a new public tender to procure security cameras and video management infrastructure.

Your response MUST:
- Be at least **1500 words** in length.
- Fully reflect the **structure, level of detail, language style**, and **subsection formatting** used in Irish public tenders (like the examples above).
- Include multiple tables (e.g., camera specs, VMS features, installation steps, SLAs, etc.).
- Use **bullet points**, **numbered sections**, **section headers**, and **clear formatting**.
- Explicitly mention relevant Irish/EU laws (e.g., GDPR, CE Marking, ISO27001, 2014/24/EU).
- Present both **mandatory** and **desirable** features, clearly marked.
- Emphasize sustainability, accessibility, cybersecurity, training, and documentation expectations.
- Cover full **lifecycle**: procurement, delivery, installation, support, exit/transition.
- **Table-layout rules:** any table must fit on an A4 page in portrait orientation with 2 cm margins;
  • limit to **max 3-4 columns**, each ≤ 6 cm wide.
  • If a cell’s text would overflow, replace it with a superscript footnote symbol (e.g. ¹) and put the full explanation in a “Table Notes” paragraph directly under the table.
  • Split very wide tables into multiple stacked tables (e.g. break after the “Verification Method” column).
- **Column & text limits:** every table may contain **max 4 columns** and **max 30 characters per cell** (including spaces).
  • Overflow → superscript note mark, explanation in “Table Notes”.
  • Split if still too wide.
  • **Do NOT reuse the sample column headings; choose headings appropriate to each table.**
  Example layout (for structure only, not wording):
  | A | B | C | D¹ |
For the “Min. Numerical Value” column, always state a concrete figure and unit (e.g., “0.01 Lux”, “IP66”); **do not use Yes/No here**—if the feature is binary, move Yes/No to a separate “Supported?” column instead.
- In every table, **do NOT leave any cell blank**.
  • If the value is the same as the row above, repeat it verbatim (e.g., write “Platform” again) or use the ditto symbol “〃”.
  • Keep the column order fixed so that each row has exactly the same number of cells.

- Insert Word-friendly formatting hints (e.g. “|---|” rows exactly match the final column count) so the table can be pasted without breaking.


Do not generate any questions. Only provide the full Requirements and Specifications section, formatted as it would appear in a formal public tender.

Start your section now.
"""
response = client.models.generate_content(
    model="gemini-2.5-flash",  # or "gemini-1.5-pro"
    contents=draft_prompt
)

# Output the generated response safely
try:
    print("📄 FINAL LONG R&S SECTION:\n")
    print(response.text)
except AttributeError:
    print(response.candidates[0].content.parts[0].text)

📄 FINAL LONG R&S SECTION:

**Dublin City Council Tender Reference: DCC-SEC-2024-001**

**REQUIREMENTS AND SPECIFICATIONS FOR THE PROVISION OF SECURITY CAMERA AND VIDEO MANAGEMENT INFRASTRUCTURE**

## 1. INTRODUCTION AND SCOPE OF REQUIREMENTS

Dublin City Council (DCC) invites tenders for the provision, installation, commissioning, and ongoing maintenance of a comprehensive security camera and video management infrastructure. This strategic procurement aims to significantly enhance public safety, deter criminal activity, and improve urban traffic flow analysis across key municipal assets and public spaces within Dublin City. The successful tenderer will be required to deliver an end-to-end solution that integrates seamlessly with Dublin City Council's existing network infrastructure and leverages current Video Management System (VMS) platforms.

The scope of this Framework Agreement encompasses the supply and deployment of approximately sixty (60) advanced security camera units, complem